# 📈 Stock Signal Analysis

### RUN Sentiment_analysis (news), earning date before running this file

### update balance sheet, once every month (limit is 12 stocks per day)

In [69]:
import pandas as pd
import numpy as np
import yfinance as yf  # For fetching stock price data
# from alpaca_setup import *  # Alpaca API setup (if needed)

import importlib
import signal_analysis_functions
importlib.reload(signal_analysis_functions)  # Reload to pick up calculate_fibonacci_levels

from signal_analysis_functions import (
    calculate_technical_indicators, 
    custom_scale, 
    generate_strict_signals, 
    rsi_signals, fi_signals_strict, 
    bollinger_signal_middle, macd_signals, 
    fibonacci_signals, weighted_signal)

import importlib
import sector_mapping

importlib.reload(sector_mapping)

stock_sector_df = sector_mapping.stock_sector_df
stock_symbols = sector_mapping.stock_symbols


# from sector_mapping import stock_sector_df, stock_symbols

In [70]:
len(stock_symbols)

93

## 1. Fetch Historical Data and Calculate Technical Indicators

Downloads 2 years of historical price data for all stocks and calculates technical indicators:

**Technical Indicators Calculated:**
- Moving Averages (MA_10, MA_30, MA_50, MA_100, MA_200)
- RSI (Relative Strength Index)
- MACD and Signal Line
- Bollinger Bands (Upper, Middle, Lower)
- Force Index (FI)
- On-Balance Volume (OBV)

**Data Period**: 2 years of daily price data for comprehensive analysis

In [71]:
## symbols to analyze
# stock_symbols is imported from sector_mapping.py
# For testing with smaller set, uncomment below:
# stock_symbols = ['AMD', 'TSLA', 'META', 'GOOGL']

all_data = pd.DataFrame()

# Loop through each stock symbol
for symbol in stock_symbols:
    # Fetch 2 years of historical data using yfinance
    stock = yf.Ticker(symbol)
    df = stock.history(period="2y").reset_index()
    
    # Remove dividend and stock split columns (not needed for technical analysis)
    df = df.drop(columns=["Dividends", "Stock Splits"], errors='ignore')
    
    # Convert Date to date format and add Symbol column
    df["Date"] = pd.to_datetime(df["Date"]).dt.date
    df["Symbol"] = symbol
    
    # Progress indicator
    print(symbol)
    
    # Calculate all technical indicators (RSI, MACD, Bollinger Bands, etc.)
    df = calculate_technical_indicators(df)
    
    # Append to master DataFrame
    all_data = pd.concat([all_data, df], ignore_index=True)

# Display last few rows to verify data
print(f"\n✅ Loaded data for {len(stock_symbols)} stocks")
print(f"   Total rows: {len(all_data)}")

all_data

AAPL
ADSK
ADBE
AMD
AMZN
ANET
APP
ARM
AVAV
ASML
AVGO
AXON
BAC
BIIB
CDNS
CDW
COF
COIN
CRWD
CRM
CRWV
CVLT
CVX
DASH
DDOG
ENPH
FLEX
FTNT
GLW
GOOGL
GTLB
HAL
HIMS
HOOD
HUBS
INTC
INTU
IONQ
IREN
IT
JOBY
KLAC
KVYO
LRCX
MCK
MRK
META
MRVL
MSFT
MU
NBIS
NET
NFLX
NOW
NSC
NVDA
ORCL
PANW
PAYX
PCTY
PLTR
POOL
PWR
RCL
REGN
RTX
SHOP
SNOW
SOFI
SOUN
TEAM
TEM
TMO
TMUS
TSLA
TTD
TYL
TWLO
UBER
UI
UNH
UPST
VEEV
VRT
WDAY
ZS
GC=F
SI=F
BTC-USD
ETH-USD
SOL-USD
HG=F
PA=F

✅ Loaded data for 93 stocks
   Total rows: 46780


,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,rsi,bb_middle,bb_std,bb_upper,bb_lower,macd,macd_signal,fi_raw,fi,obv_std,obv,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%
0,2024-01-10,182.63,184.66,182.20,184.45,46792900,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.33,277.84,272.61,269.38,266.77,264.16,260.93,255.70
1,2024-01-11,184.79,185.30,181.90,183.85,49128400,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,-0.05,-0.01,-29202222.02,13.81,-49128400,0.31,277.84,272.61,269.38,266.77,264.16,260.93,255.70
2,2024-01-12,184.32,184.99,183.46,184.18,40477800,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,-0.06,-0.02,-23139873.20,14.90,-8650600,0.32,277.84,272.61,269.38,266.77,264.16,260.93,255.70
3,2024-01-16,180.46,182.54,179.24,181.91,65603000,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,-0.25,-0.06,-41094890.49,11.68,-74253600,0.30,277.84,272.61,269.38,266.77,264.16,260.93,255.70
4,2024-01-17,179.57,181.22,178.61,180.97,47317400,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,-0.47,-0.15,-41585788.70,11.59,-121571000,0.29,277.84,272.61,269.38,266.77,264.16,260.93,255.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46775,2026-01-05,1700.00,1768.50,1680.00,1742.20,21,PA=F,1760.64,1583.58,1524.42,1385.99,1218.82,58.38,1660.02,145.77,1951.56,1368.49,67.01,76.84,-940.38,-81.76,-15430,0.08,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20
46776,2026-01-06,1865.60,1865.60,1865.60,1865.60,0,PA=F,1772.30,1599.68,1532.31,1393.43,1223.45,59.71,1679.68,145.65,1970.98,1388.39,74.83,76.44,-806.04,-81.52,-15430,0.08,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20
46777,2026-01-07,1767.20,1767.20,1767.20,1767.20,0,PA=F,1769.68,1612.47,1538.38,1399.66,1227.53,54.78,1693.86,139.22,1972.30,1415.43,72.25,75.60,-690.89,-81.30,-15430,0.08,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20
46778,2026-01-08,1781.10,1781.10,1781.10,1781.10,0,PA=F,1755.93,1625.07,1545.69,1406.40,1231.70,53.40,1707.35,133.56,1974.47,1440.23,70.51,74.58,-592.19,-81.12,-15430,0.08,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20


## 2. Generate Trading Signals

Applies all signal generation functions to create buy/sell signals from technical indicators.

**Signal Generation Steps:**
1. **Scale Data**: Normalize Force Index, Volume, and OBV within each stock group
2. **Moving Average Signals**: Generate signals from MA crossovers and price position
3. **RSI Signals**: Generate signals based on RSI overbought/oversold conditions
4. **Force Index Signals**: Generate signals from volume-weighted price momentum
5. **Bollinger Band Signals**: Generate signals when price touches bands
6. **MACD Signals**: Generate signals from MACD line crossovers
7. **Oversold Reversal Signals**: Detect significant price drops for reversal opportunities
8. **Weighted Combination**: Combine all signals into a single `combined_signal` score

**Output**: DataFrame with `combined_signal` column containing weighted score for each day

In [72]:
# STEP 1: SCALE DATA WITHIN EACH STOCK GROUP
df_scaled = all_data.copy()
for col in ['fi', 'Volume', 'obv']:
    df_scaled[col] = df_scaled.groupby('Symbol')[col].transform(custom_scale)

df = df_scaled.copy()

# STEP 2: GENERATE MOVING AVERAGE SIGNALS
df = generate_strict_signals(df)

# STEP 3: GENERATE RSI SIGNALS
df = df.groupby('Symbol', group_keys=False).apply(rsi_signals)

# STEP 4: GENERATE FORCE INDEX SIGNALS
df = df.groupby('Symbol', group_keys=False).apply(fi_signals_strict)

# STEP 5: GENERATE BOLLINGER BAND SIGNALS
df = bollinger_signal_middle(df)

# STEP 6: GENERATE MACD SIGNALS
df = df.groupby('Symbol', group_keys=False).apply(macd_signals)

# STEP 7: GENERATE FIBONACCI SIGNALS
df = fibonacci_signals(df)

df = weighted_signal(df)

# Fill NaN values with 0 and convert to integer
df["combined_signal"] = df["combined_signal"].fillna(0).astype("int")


# FINALIZE DATA FORMAT

# Ensure Date is datetime format for proper sorting and filtering
df['Date'] = pd.to_datetime(df['Date'])
df = df.reset_index(drop=True)

print(f"✅ Generated signals for {df['Symbol'].nunique()} stocks")
print(f"   Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"   Combined signal range: {df['combined_signal'].min()} to {df['combined_signal'].max()}")
df.tail(5)

✅ Generated signals for 93 stocks
   Date range: 2024-01-09 00:00:00 to 2026-01-09 00:00:00
   Combined signal range: -88 to 88


,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,rsi,macd,macd_signal,fi_raw,fi,obv_std,obv,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%,signal_ma_10,signal_ma_30,signal_ma_50,signal_ma_100,signal_ma_200,obv_slope,signal_obv,rsi_signal,fi_signal,bb_signal,macd_trade,fib_signal,combined_signal
46775,2026-01-05,1700.00,1768.50,1680.00,1742.20,0.27,PA=F,1760.64,1583.58,1524.42,1385.99,1218.82,58.38,67.01,76.84,-940.38,-32.64,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,-1,1,1,1,1,0.13,1,0,-1,1,0,0,44
46776,2026-01-06,1865.60,1865.60,1865.60,1865.60,0.00,PA=F,1772.30,1599.68,1532.31,1393.43,1223.45,59.71,74.83,76.44,-806.04,-31.73,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,1,1,1,1,1,0.13,1,0,0,1,0,0,58
46777,2026-01-07,1767.20,1767.20,1767.20,1767.20,0.00,PA=F,1769.68,1612.47,1538.38,1399.66,1227.53,54.78,72.25,75.60,-690.89,-30.95,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,0,1,1,1,1,0.13,1,0,-1,1,0,0,48
46778,2026-01-08,1781.10,1781.10,1781.10,1781.10,0.00,PA=F,1755.93,1625.07,1545.69,1406.40,1231.70,53.40,70.51,74.58,-592.19,-30.28,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,1,1,1,1,1,0.00,0,0,0,1,0,1,62
46779,2026-01-09,1830.00,1937.00,1793.00,1874.00,100.00,PA=F,1766.70,1640.80,1555.11,1413.92,1236.30,54.64,75.76,74.82,98165.50,NaN,-7995,48.83,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,1,1,1,1,1,47.50,1,0,0,1,1,0,72


In [73]:
df[df["Symbol"] == "ANET"]

,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,rsi,macd,macd_signal,fi_raw,fi,obv_std,obv,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%,signal_ma_10,signal_ma_30,signal_ma_50,signal_ma_100,signal_ma_200,obv_slope,signal_obv,rsi_signal,fi_signal,bb_signal,macd_trade,fib_signal,combined_signal
2510,2024-01-10,61.88,62.81,61.52,62.31,14.01,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,0,14.62,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,-80.39,-1,0,0,0,0,0,5
2511,2024-01-11,62.49,63.04,61.92,62.94,9.36,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.05,0.01,3154580.27,27.32,5027200,15.71,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,-82.05,-1,0,0,0,0,0,5
2512,2024-01-12,63.00,63.68,62.81,63.01,9.09,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.09,0.03,2754977.20,26.92,9956400,16.77,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,-83.23,-1,0,0,0,0,0,5
2513,2024-01-16,63.11,64.30,62.84,63.31,16.90,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.15,0.05,2702100.27,26.87,17712000,18.45,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,3.83,1,0,1,0,0,0,22
2514,2024-01-17,62.72,62.72,61.31,62.25,11.15,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.11,0.06,1450803.17,25.65,12038000,17.22,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,1.51,1,0,1,0,0,0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,2026-01-05,138.34,141.41,134.51,137.19,22.26,ANET,132.42,128.56,134.39,138.50,116.72,72.13,0.66,-0.48,8066225.60,32.13,393724400,99.65,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,1,0,0,2.28,1,0,1,0,0,1,42
3008,2026-01-06,137.90,137.93,132.50,132.58,17.59,ANET,132.57,128.99,133.99,138.44,116.96,60.60,0.66,-0.25,1642701.25,25.83,385720400,97.92,141.41,138.92,137.39,136.15,134.90,133.37,130.88,0,1,0,-1,0,1.38,1,0,1,0,0,0,9
3009,2026-01-07,130.01,131.65,128.64,130.08,16.34,ANET,132.50,129.41,133.51,138.38,117.20,55.84,0.45,-0.11,-1288648.93,22.96,378169700,96.29,141.41,138.92,137.39,136.15,134.90,133.37,130.88,-1,0,0,-1,0,-1.27,-1,0,0,-1,0,0,-27
3010,2026-01-08,129.37,129.60,122.81,123.72,20.19,ANET,131.74,129.46,132.85,138.24,117.38,51.87,-0.22,-0.13,-9231001.58,15.17,369225500,94.36,141.41,138.92,137.39,136.15,134.90,133.37,130.88,-1,-1,0,-1,0,-5.29,-1,0,0,-1,-1,0,-48


## 2.2. Integrate Sentiment and Fundamental Analysis

Enhances trading signals by incorporating:
1. **News Sentiment Scores**: Market sentiment from news analysis
2. **Fundamental Analysis Scores**: Fundamental weight from balance sheet analysis

**Process:**
1. Load sentiment scores from `weighted_sentiment.csv`
2. Load fundamental weights from `fundamental_weight_analysis.xlsx`
3. Merge both with trading signals
4. Calculate new combined score: `combined_signal + SentimentScore + Fundamental_Weight`
5. Regenerate trade signals based on adjusted score
6. Filter out NEUTRAL signals (only keep BUY/SELL)

**Rationale**: Combines technical, sentiment, and fundamental analysis for more robust trading decisions.

In [74]:
# ============================================================================
# LOAD SENTIMENT AND FUNDAMENTAL ANALYSIS DATA
# ============================================================================
import os
REPORTS_DIR = os.path.join(os.getcwd(), "Reports")

# Load news sentiment scores (from sentiment_analysis.ipynb)
# SentimentScore: Positive = bullish sentiment, Negative = bearish sentiment
weighted_sentiment = pd.read_csv(os.path.join(REPORTS_DIR, "weighted_sentiment.csv"))

# Load fundamental analysis weights (from Balance Sheet3.ipynb)
# Fundamental_Weight: Scaled -10 to +10, positive = strong fundamentals
weighted_balance = pd.read_csv(os.path.join(REPORTS_DIR, "balance_sheet_weights.csv"))
weighted_balance = weighted_balance[["Symbol", "Fundamental_Weight"]]

print(f"✅ Loaded sentiment scores for {len(weighted_sentiment)} stocks")
print(f"✅ Loaded balance sheet weights for {len(weighted_balance)} stocks")

✅ Loaded sentiment scores for 89 stocks
✅ Loaded balance sheet weights for 97 stocks


In [75]:
buy_sell_senti_df = pd.merge(df, weighted_sentiment, on=["Symbol"], how='left')

balance_senti_df = pd.merge(buy_sell_senti_df, weighted_balance, on=["Symbol"], how='left')

balance_senti_df = balance_senti_df.fillna(0)

balance_senti_df["combined_signal"] = (
    balance_senti_df["combined_signal"] + 
    balance_senti_df["SentimentScore"] + 
    balance_senti_df["Fundamental_Weight"]
)


print(f"✅ Merged sentiment and fundamental data")
print(f"   New Score range: {balance_senti_df['combined_signal'].min():.2f} to {balance_senti_df['combined_signal'].max():.2f}")
# balance_senti_df = balance_senti_df.drop(columns = ["SentimentScore", "Fundamental_Weight"])
balance_senti_df

✅ Merged sentiment and fundamental data
   New Score range: -83.73 to 105.24


,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,rsi,macd,macd_signal,fi_raw,fi,obv_std,obv,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%,signal_ma_10,signal_ma_30,signal_ma_50,signal_ma_100,signal_ma_200,obv_slope,signal_obv,rsi_signal,fi_signal,bb_signal,macd_trade,fib_signal,combined_signal,SentimentScore,Fundamental_Weight
0,2024-01-10,182.63,184.66,182.20,184.45,9.60,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,0,31.54,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,14.75,4.75,0.00
1,2024-01-11,184.79,185.30,181.90,183.85,10.38,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.05,-0.01,-29202222.02,13.77,-49128400,29.88,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,14.75,4.75,0.00
2,2024-01-12,184.32,184.99,183.46,184.18,7.50,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.06,-0.02,-23139873.20,14.86,-8650600,31.25,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,14.75,4.75,0.00
3,2024-01-16,180.46,182.54,179.24,181.91,15.86,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.25,-0.06,-41094890.49,11.64,-74253600,29.03,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,-2.51,-1,0,1,0,0,0,16.75,4.75,0.00
4,2024-01-17,179.57,181.22,178.61,180.97,9.78,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.47,-0.15,-41585788.70,11.55,-121571000,27.43,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,-2.45,-1,0,1,0,0,0,16.75,4.75,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46775,2026-01-05,1700.00,1768.50,1680.00,1742.20,0.27,PA=F,1760.64,1583.58,1524.42,1385.99,1218.82,58.38,67.01,76.84,-940.38,-32.64,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,-1,1,1,1,1,0.13,1,0,-1,1,0,0,44.00,0.00,0.00
46776,2026-01-06,1865.60,1865.60,1865.60,1865.60,0.00,PA=F,1772.30,1599.68,1532.31,1393.43,1223.45,59.71,74.83,76.44,-806.04,-31.73,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,1,1,1,1,1,0.13,1,0,0,1,0,0,58.00,0.00,0.00
46777,2026-01-07,1767.20,1767.20,1767.20,1767.20,0.00,PA=F,1769.68,1612.47,1538.38,1399.66,1227.53,54.78,72.25,75.60,-690.89,-30.95,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,0,1,1,1,1,0.13,1,0,-1,1,0,0,48.00,0.00,0.00
46778,2026-01-08,1781.10,1781.10,1781.10,1781.10,0.00,PA=F,1755.93,1625.07,1545.69,1406.40,1231.70,53.40,70.51,74.58,-592.19,-30.28,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,1,1,1,1,1,0.00,0,0,0,1,0,1,62.00,0.00,0.00


## 3. Add Sector Performance and Adjust Signals

Incorporates sector-level performance data to adjust individual stock signals.

**Process:**
1. **Fetch Sector Data**: Downloads sector ETF performance data using yfinance
2. **Calculate Sector Returns**: Computes monthly returns for each sector
3. **Merge Sector Data**: Adds sector information and returns to stock data
4. **Adjust Combined Signal**: Adds sector weight (2x monthly return) to combined signal
5. **Generate Trade Signals**: Converts adjusted signal to BUY/SELL/HOLD based on thresholds:
   - **BUY**: `combined_signal > 60`
   - **SELL**: `combined_signal < -20`
   - **HOLD**: All other values

**Rationale**: Stocks in outperforming sectors get boosted signals, while stocks in underperforming sectors get reduced signals.

In [76]:
# Mapping of sector ETF symbols to sector names
sector_etfs = {
    "XLC": "Communication Services",
    "XLY": "Consumer Discretionary",
    "XLP": "Consumer Staples",
    "XLE": "Energy",
    "XLF": "Financials",
    "XLV": "Health Care",
    "XLI": "Industrials",
    "XLB": "Materials",
    "XLRE": "Real Estate",
    "XLK": "Technology",
    "XLU": "Utilities"
}

# Download 2 weeks of sector ETF data to calculate recent sector performance
data = yf.download(list(sector_etfs.keys()), period="1wk")["Close"]

# Compute total return over the period (first day to last day)
# Positive returns = outperforming sector (boosts stock signals)
# Negative returns = underperforming sector (reduces stock signals)
returns = (data.iloc[-1] / data.iloc[0] - 1).sort_values(ascending=False)

# Create DataFrame with sector names and monthly return percentages
sector_data = pd.DataFrame({
    "Sector": [sector_etfs[s] for s in returns.index],
    "Monthly Return (%)": (returns.values * 100).round(2)
})

sector_data

[*********************100%***********************]  11 of 11 completed


,Sector,Monthly Return (%)
0,Materials,3.61
1,Consumer Discretionary,3.42
2,Consumer Staples,2.46
3,Health Care,1.46
4,Industrials,1.30
5,Technology,1.06
6,Real Estate,0.17
7,Communication Services,0.14
8,Utilities,-0.44
9,Energy,-0.47


In [77]:
df = balance_senti_df.copy() ### reassigning df 

# ============================================================================
# MERGE SECTOR DATA WITH STOCK DATA
# ============================================================================
sector_analysis = pd.merge(stock_sector_df, sector_data, on="Sector", how="left")

sector_analysis["Sector Score"] = sector_analysis["Monthly Return (%)"] * 3

# Merge sector analysis with main dataframe
df = pd.merge(df, sector_analysis, on=["Symbol"], how='left')
df['combined_signal'] = df["combined_signal"].fillna(0).astype(int)
df["Sector Score"] = df["Sector Score"].fillna(0)

df["new_signal"] = df["combined_signal"] + df["Sector Score"]

# Clean up: remove old combined_signal and temporary columns
df = df.drop(columns=["combined_signal", "Sector Score"])
df = df.rename(columns={'new_signal': 'combined_signal'})

# ============================================================================
# GENERATE FINAL TRADE SIGNALS
# ============================================================================
# Convert combined_signal to BUY/SELL/HOLD based on thresholds:
# - BUY: combined_signal > 36.7 (strong positive signal)
# - SELL: combined_signal < -12 (strong negative signal)
# - HOLD: All other values (neutral or weak signals)
conditions = [df['combined_signal'] > 50, df['combined_signal'] < -15]
choices = ['BUY', 'SELL']
df['final_trade'] = np.select(conditions, choices, default='HOLD')
df

,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,rsi,macd,macd_signal,fi_raw,fi,obv_std,obv,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%,signal_ma_10,signal_ma_30,signal_ma_50,signal_ma_100,signal_ma_200,obv_slope,signal_obv,rsi_signal,fi_signal,bb_signal,macd_trade,fib_signal,SentimentScore,Fundamental_Weight,Sector,Company Name,Monthly Return (%),combined_signal,final_trade
0,2024-01-10,182.63,184.66,182.20,184.45,9.60,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,0,31.54,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,17.18,HOLD
1,2024-01-11,184.79,185.30,181.90,183.85,10.38,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.05,-0.01,-29202222.02,13.77,-49128400,29.88,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,17.18,HOLD
2,2024-01-12,184.32,184.99,183.46,184.18,7.50,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.06,-0.02,-23139873.20,14.86,-8650600,31.25,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,17.18,HOLD
3,2024-01-16,180.46,182.54,179.24,181.91,15.86,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.25,-0.06,-41094890.49,11.64,-74253600,29.03,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,-2.51,-1,0,1,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,19.18,HOLD
4,2024-01-17,179.57,181.22,178.61,180.97,9.78,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.47,-0.15,-41585788.70,11.55,-121571000,27.43,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,-2.45,-1,0,1,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,19.18,HOLD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46775,2026-01-05,1700.00,1768.50,1680.00,1742.20,0.27,PA=F,1760.64,1583.58,1524.42,1385.99,1218.82,58.38,67.01,76.84,-940.38,-32.64,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,-1,1,1,1,1,0.13,1,0,-1,1,0,0,0.00,0.00,Commodities,Palladium,NaN,44.00,HOLD
46776,2026-01-06,1865.60,1865.60,1865.60,1865.60,0.00,PA=F,1772.30,1599.68,1532.31,1393.43,1223.45,59.71,74.83,76.44,-806.04,-31.73,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,1,1,1,1,1,0.13,1,0,0,1,0,0,0.00,0.00,Commodities,Palladium,NaN,58.00,BUY
46777,2026-01-07,1767.20,1767.20,1767.20,1767.20,0.00,PA=F,1769.68,1612.47,1538.38,1399.66,1227.53,54.78,72.25,75.60,-690.89,-30.95,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,0,1,1,1,1,0.13,1,0,-1,1,0,0,0.00,0.00,Commodities,Palladium,NaN,48.00,HOLD
46778,2026-01-08,1781.10,1781.10,1781.10,1781.10,0.00,PA=F,1755.93,1625.07,1545.69,1406.40,1231.70,53.40,70.51,74.58,-592.19,-30.28,-15430,1.33,1865.60,1842.38,1828.01,1816.40,1804.79,1790.42,1767.20,1,1,1,1,1,0.00,0,0,0,1,0,1,0.00,0.00,Commodities,Palladium,NaN,62.00,BUY


## 4. Calculate Buy/Sell Streaks and Export to Tableau

Calculates consecutive BUY and SELL signal streaks for each stock, then exports full historical data for Tableau visualization.

**Streak Calculation:**
- **Buy Streak**: Consecutive days with positive `combined_signal` (BUY signals)
- **Sell Streak**: Consecutive days with negative `combined_signal` (SELL signals)
- Streaks help identify momentum and signal strength

**Tableau Export:**
- Exports all historical data with technical indicators
- Adds `trade_value` column (+50 for BUY, -50 for SELL, 0 for HOLD) for visualization
- Removes unnecessary columns for cleaner Tableau dashboard

In [78]:
# ============================================================================
# CALCULATE BUY/SELL STREAKS
# ============================================================================
# Streaks track consecutive days of buy or sell signals
# Higher streaks indicate stronger momentum and signal persistence

# Sort data by Symbol and Date for proper streak calculation
trade_df = df.sort_values(by=["Symbol", "Date"], ascending=[True, True])

buy_streak_list = []
sell_streak_list = []

# Calculate streaks for each stock separately
for symbol, group in trade_df.groupby("Symbol"):
    buy_count = sell_count = 0
    buy_mode = sell_mode = False
    group_buy_streak = []
    group_sell_streak = []

    # Process each day's signal for this stock
    for signal in group["combined_signal"]:
        # ====================================================================
        # BUY STREAK LOGIC
        # ====================================================================
        if signal > 50:  # Strong buy signal - start or continue buy streak
            buy_mode = True
            sell_mode = False
            buy_count += 1
            sell_count = 0
        elif buy_mode and signal >= 20:  # Continue buy streak (moderate positive)
            buy_count += 1
            sell_count = 0
        # ====================================================================
        # SELL STREAK LOGIC
        # ====================================================================
        elif signal < -15:  # Strong sell signal - start sell streak
            buy_mode = False
            sell_mode = True
            buy_count = 0
            sell_count += 1
        elif sell_mode and signal < 10:  # Continue sell streak (must stay negative)
            sell_count += 1
            buy_count = 0
        # ====================================================================
        # NEUTRAL - RESET STREAKS
        # ====================================================================
        else:
            # Signal is neutral or mode changed - reset both
            buy_mode = sell_mode = False
            buy_count = sell_count = 0

        group_buy_streak.append(buy_count)
        group_sell_streak.append(sell_count) 

    # Add streaks for this stock to master lists
    buy_streak_list.extend(group_buy_streak)
    sell_streak_list.extend(group_sell_streak)

# Add streak columns to dataframe
trade_df["Buy Streak"] = buy_streak_list
trade_df["Sell Streak"] = sell_streak_list

print(f"✅ Calculated streaks for {trade_df['Symbol'].nunique()} stocks")
print(f"   Max Buy Streak: {trade_df['Buy Streak'].max()}")
print(f"   Max Sell Streak: {trade_df['Sell Streak'].max()}")
trade_df

✅ Calculated streaks for 93 stocks
   Max Buy Streak: 137
   Max Sell Streak: 109


,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,rsi,macd,macd_signal,fi_raw,fi,obv_std,obv,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%,signal_ma_10,signal_ma_30,signal_ma_50,signal_ma_100,signal_ma_200,obv_slope,signal_obv,rsi_signal,fi_signal,bb_signal,macd_trade,fib_signal,SentimentScore,Fundamental_Weight,Sector,Company Name,Monthly Return (%),combined_signal,final_trade,Buy Streak,Sell Streak
0,2024-01-10,182.63,184.66,182.20,184.45,9.60,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,0,31.54,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,17.18,HOLD,0,0
1,2024-01-11,184.79,185.30,181.90,183.85,10.38,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.05,-0.01,-29202222.02,13.77,-49128400,29.88,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,17.18,HOLD,0,0
2,2024-01-12,184.32,184.99,183.46,184.18,7.50,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.06,-0.02,-23139873.20,14.86,-8650600,31.25,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,0.00,0,0,0,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,17.18,HOLD,0,0
3,2024-01-16,180.46,182.54,179.24,181.91,15.86,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.25,-0.06,-41094890.49,11.64,-74253600,29.03,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,-2.51,-1,0,1,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,19.18,HOLD,0,0
4,2024-01-17,179.57,181.22,178.61,180.97,9.78,AAPL,0.00,0.00,0.00,0.00,0.00,50.00,-0.47,-0.15,-41585788.70,11.55,-121571000,27.43,277.84,272.61,269.38,266.77,264.16,260.93,255.70,1,1,0,0,0,-2.45,-1,0,1,0,0,0,4.75,0.00,Technology,Apple Inc.,1.06,19.18,HOLD,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42560,2026-01-05,221.70,227.70,221.63,222.76,6.73,ZS,227.77,241.64,272.24,280.43,269.36,31.96,-12.22,-13.65,-2546076.95,8.41,39118200,66.44,231.48,228.51,226.67,225.18,223.70,221.86,218.89,-1,-1,-1,-1,-1,-1.30,-1,0,0,0,0,0,2.73,-10.52,Technology,"Zscaler, Inc.",1.06,-52.82,SELL,0,32
42561,2026-01-06,222.61,223.43,218.89,222.03,8.70,ZS,226.72,239.72,270.31,279.88,269.45,37.16,-11.85,-13.29,-2342450.17,8.80,37583000,64.99,231.48,228.51,226.67,225.18,223.70,221.86,218.89,-1,-1,-1,-1,-1,-1.81,-1,0,0,0,0,0,2.73,-10.52,Technology,"Zscaler, Inc.",1.06,-52.82,SELL,0,33
42562,2026-01-07,223.01,231.48,222.18,231.16,14.02,ZS,226.63,238.26,268.47,279.48,269.58,47.92,-10.70,-12.77,777358.78,14.83,39718400,67.01,231.48,228.51,226.67,225.18,223.70,221.86,218.89,1,-1,-1,-1,-1,1.81,1,0,1,1,0,-1,2.73,-10.52,Technology,"Zscaler, Inc.",1.06,-27.82,SELL,0,34
42563,2026-01-08,230.65,230.79,219.38,220.32,12.88,ZS,225.73,236.26,266.35,278.93,269.63,42.96,-10.53,-12.32,-2441054.86,8.61,37711800,65.12,231.48,228.51,226.67,225.18,223.70,221.86,218.89,-1,-1,-1,-1,-1,-1.33,-1,0,0,0,0,0,2.73,-10.52,Technology,"Zscaler, Inc.",1.06,-52.82,SELL,0,35


In [79]:
trade_df[trade_df["Symbol"] == "ANET"]

,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,rsi,macd,macd_signal,fi_raw,fi,obv_std,obv,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%,signal_ma_10,signal_ma_30,signal_ma_50,signal_ma_100,signal_ma_200,obv_slope,signal_obv,rsi_signal,fi_signal,bb_signal,macd_trade,fib_signal,SentimentScore,Fundamental_Weight,Sector,Company Name,Monthly Return (%),combined_signal,final_trade,Buy Streak,Sell Streak
2510,2024-01-10,61.88,62.81,61.52,62.31,14.01,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.00,0.00,0.00,0.00,0,14.62,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,-80.39,-1,0,0,0,0,0,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,17.18,HOLD,0,0
2511,2024-01-11,62.49,63.04,61.92,62.94,9.36,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.05,0.01,3154580.27,27.32,5027200,15.71,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,-82.05,-1,0,0,0,0,0,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,17.18,HOLD,0,0
2512,2024-01-12,63.00,63.68,62.81,63.01,9.09,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.09,0.03,2754977.20,26.92,9956400,16.77,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,-83.23,-1,0,0,0,0,0,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,17.18,HOLD,0,0
2513,2024-01-16,63.11,64.30,62.84,63.31,16.90,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.15,0.05,2702100.27,26.87,17712000,18.45,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,3.83,1,0,1,0,0,0,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,34.18,HOLD,0,0
2514,2024-01-17,62.72,62.72,61.31,62.25,11.15,ANET,0.00,0.00,0.00,0.00,0.00,50.00,0.11,0.06,1450803.17,25.65,12038000,17.22,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,0,0,0,1.51,1,0,1,0,0,0,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,34.18,HOLD,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3007,2026-01-05,138.34,141.41,134.51,137.19,22.26,ANET,132.42,128.56,134.39,138.50,116.72,72.13,0.66,-0.48,8066225.60,32.13,393724400,99.65,141.41,138.92,137.39,136.15,134.90,133.37,130.88,1,1,1,0,0,2.28,1,0,1,0,0,1,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,54.18,BUY,1,0
3008,2026-01-06,137.90,137.93,132.50,132.58,17.59,ANET,132.57,128.99,133.99,138.44,116.96,60.60,0.66,-0.25,1642701.25,25.83,385720400,97.92,141.41,138.92,137.39,136.15,134.90,133.37,130.88,0,1,0,-1,0,1.38,1,0,1,0,0,0,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,21.18,HOLD,2,0
3009,2026-01-07,130.01,131.65,128.64,130.08,16.34,ANET,132.50,129.41,133.51,138.38,117.20,55.84,0.45,-0.11,-1288648.93,22.96,378169700,96.29,141.41,138.92,137.39,136.15,134.90,133.37,130.88,-1,0,0,-1,0,-1.27,-1,0,0,-1,0,0,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,-13.82,HOLD,0,0
3010,2026-01-08,129.37,129.60,122.81,123.72,20.19,ANET,131.74,129.46,132.85,138.24,117.38,51.87,-0.22,-0.13,-9231001.58,15.17,369225500,94.36,141.41,138.92,137.39,136.15,134.90,133.37,130.88,-1,-1,0,-1,0,-5.29,-1,0,0,-1,-1,0,4.00,5.34,Technology,"Arista Networks, Inc.",1.06,-34.82,SELL,0,1


#### Save file for Tableau Analysis

In [80]:
# ============================================================================
# PREPARE DATA FOR TABLEAU EXPORT
# ============================================================================
# Remove columns that are not needed for Tableau visualization
col_drop_tableau = ['signal_ma_10', 'signal_ma_30', 'signal_ma_50', 'signal_ma_100', 'signal_ma_200', 
                    "rsi_signal", "fi_signal", "bb_signal", "macd_trade", 'fi_raw', 'signal_obv',
                    'obv_std', 'obv', 'obv_slope', 'Company Name', 'Sector']
tableau_df = trade_df.drop(columns=col_drop_tableau)

# ============================================================================
# ADD TRADE VALUE COLUMN FOR VISUALIZATION
# ============================================================================
# Create trade_value column for Tableau bar charts:
# - BUY signals: +50 (positive bar)
# - SELL signals: -50 (negative bar)
# - HOLD signals: 0 (no bar)
tableau_df["trade_value"] = np.where(
    tableau_df["final_trade"] == "BUY", 50,
    np.where(tableau_df["final_trade"] == "SELL", -50, 0)
)

# ============================================================================
# FORMAT DATA FOR TABLEAU
# ============================================================================
# Ensure Date is datetime format
tableau_df["Date"] = pd.to_datetime(tableau_df["Date"], errors='coerce')

# Sort by Symbol and Date (descending - most recent first)
tableau_df = tableau_df.sort_values(by=["Symbol", "Date"], ascending=[True, False])

# Ensure Symbol is string type
tableau_df["Symbol"] = tableau_df["Symbol"].astype(str)

# Round all numerical columns to 2 decimal places for cleaner display
tableau_df = tableau_df.apply(lambda x: x.round(2) if x.dtype in ['float64', 'int64'] else x)

# Rename columns for better readability in Tableau
tableau_df = tableau_df.rename(columns={
    'rsi': 'RSI Options Rate',
    'macd_signal': 'MACD Signal',
    'Score': 'Score Trend'
})

# Remove final_trade column (trade_value already represents this)
# tableau_df = tableau_df.drop(columns="final_trade")

# ============================================================================
# EXPORT DATA - CSV if symbols > 100, Excel otherwise
# ============================================================================
import os
REPORTS_DIR = os.path.join(os.getcwd(), "Reports")
os.makedirs(REPORTS_DIR, exist_ok=True)

# Count unique symbols to determine output format
num_symbols = tableau_df['Symbol'].nunique() if not tableau_df.empty else 0

try:
    if tableau_df.empty:
        print("⚠️ No data to export")
    else:
        # Save CSV if symbols > 100, Excel otherwise
        if num_symbols > 50:
            csv_file = os.path.join(REPORTS_DIR, "signal_analysis.csv")
            tableau_df.to_csv(csv_file, index=False)
            print(f"✅ Data saved successfully to {csv_file} (CSV format)")
            print(f"   Format: CSV (symbols > 100)")
        else:
            excel_file = os.path.join(REPORTS_DIR, "signal_analysis.xlsx")
            tableau_df.to_excel(excel_file, index=False)
            print(f"✅ Data saved successfully to {excel_file} (Excel format)")
            print(f"   Format: Excel (symbols <= 100)")
        
        print(f"   Unique symbols: {num_symbols}")
        print(f"   Total rows: {len(tableau_df)}")
        print(f"   Total columns: {len(tableau_df.columns)}")
        print(f"   Date range: {tableau_df['Date'].min()} to {tableau_df['Date'].max()}")
except Exception as e:
    print(f"❌ Error saving data: {e}")

✅ Data saved successfully to /Users/drcnavad/Desktop/Stock Analysis/Reports/signal_analysis.csv (CSV format)
   Format: CSV (symbols > 100)
   Unique symbols: 93
   Total rows: 46780
   Total columns: 32
   Date range: 2024-01-09 00:00:00 to 2026-01-09 00:00:00


In [81]:
tableau_df[tableau_df["Symbol"] == "BTC-USD"]

,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,RSI Options Rate,macd,MACD Signal,fi,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%,fib_signal,SentimentScore,Fundamental_Weight,Monthly Return (%),combined_signal,final_trade,Buy Streak,Sell Streak,trade_value
44306,2026-01-09,91019.59,91814.73,89628.57,90559.36,16.94,BTC-USD,90870.83,89005.38,89280.94,98840.75,106421.81,62.39,553.84,131.57,-11.35,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,1,-1.67,0.00,NaN,-8.00,HOLD,0,0,0
44305,2026-01-08,91309.64,91485.85,89233.88,91027.12,19.17,BTC-USD,90657.91,89054.10,89202.39,99121.64,106496.90,64.88,603.16,26.00,-10.00,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,1,-1.67,0.00,NaN,-1.00,HOLD,0,0,0
44304,2026-01-07,93727.47,93738.79,90601.80,91308.05,19.79,BTC-USD,90269.01,89109.59,89211.17,99351.93,106546.70,64.40,609.71,-118.29,-8.90,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,1,-1.67,0.00,NaN,1.00,HOLD,0,0,0
44303,2026-01-06,93876.95,94395.30,91286.55,93729.03,24.99,BTC-USD,89921.79,89087.33,89243.98,99582.86,106601.45,79.75,580.60,-300.29,-0.21,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,1,-1.67,0.00,NaN,10.00,HOLD,0,0,0
44302,2026-01-05,91414.62,94762.07,91414.62,93882.55,25.54,BTC-USD,89329.10,88976.55,89211.28,99766.79,106649.35,73.37,288.50,-520.52,2.19,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,1,-1.67,0.00,NaN,1.00,HOLD,0,69,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43579,2024-01-13,42799.45,43234.66,42464.14,42842.38,6.53,BTC-USD,0.00,0.00,0.00,0.00,0.00,50.00,-437.99,-113.48,-13.05,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,0,-1.67,0.00,NaN,3.00,HOLD,0,0,0
43578,2024-01-12,46354.79,46498.14,41903.77,42853.17,19.71,BTC-USD,0.00,0.00,0.00,0.00,0.00,50.00,-225.32,-32.35,-13.40,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,0,-1.67,0.00,NaN,-9.00,HOLD,0,0,0
43577,2024-01-11,46656.07,48969.37,45678.64,46368.59,21.17,BTC-USD,0.00,0.00,0.00,0.00,0.00,50.00,48.32,15.89,-1.71,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,0,-1.67,0.00,NaN,3.00,HOLD,0,0,0
43576,2024-01-10,46121.54,47647.22,44483.15,46627.78,23.65,BTC-USD,0.00,0.00,0.00,0.00,0.00,50.00,38.93,7.79,0.75,94762.07,93457.42,92650.30,91997.97,91345.65,90538.53,89233.88,0,-1.67,0.00,NaN,3.00,HOLD,0,0,0


#### Test for sending Whatsapp Snapshot

In [82]:
REPORTS_DIR = os.path.join(os.getcwd(), "Reports")
earnings_file = os.path.join(REPORTS_DIR, "earnings_date.csv")
earnings_df = pd.read_csv(earnings_file)

# Convert Earnings Date to datetime
earnings_df['Earnings Date'] = pd.to_datetime(earnings_df['Earnings Date'], errors='coerce')

# Get current date from x (use the Date column)
current_date = pd.to_datetime(tableau_df['Date'].iloc[0]) if 'Date' in tableau_df.columns else pd.Timestamp.now()

# Filter for future earnings dates only (estimated/predicted earnings)
future_earnings = earnings_df[earnings_df['Earnings Date'] >= current_date].copy()

# Filter earnings dates that are within ±3 days of current date
earnings_window = future_earnings[
    (future_earnings['Earnings Date'] >= current_date - pd.Timedelta(days=3)) &
    (future_earnings['Earnings Date'] <= current_date + pd.Timedelta(days=3))
].copy()

# Get symbols with upcoming earnings
symbols_with_earnings = set(earnings_window['Symbol'].unique())
print(f"Earnings nearby: {symbols_with_earnings}")

# earning_mask = tableau_df['Symbol'].isin(symbols_with_earnings)

# # Replace BUY and SELL with EARNING for stocks with upcoming earnings
# # Both BUY and SELL signals are changed to avoid trading around earnings announcements
buy_earnings_mask = (tableau_df['Symbol'].isin(symbols_with_earnings)) & (tableau_df['final_trade'] == 'BUY')
sell_earnings_mask = (tableau_df['Symbol'].isin(symbols_with_earnings)) & (tableau_df['final_trade'] == 'SELL')

tableau_df.loc[buy_earnings_mask, 'final_trade'] = 'EARNING'
tableau_df.loc[sell_earnings_mask, 'final_trade'] = 'EARNING'

tableau_df

Earnings nearby: set()


,Date,Open,High,Low,Close,Volume,Symbol,ma_10,ma_30,ma_50,ma_100,ma_200,RSI Options Rate,macd,MACD Signal,fi,fib_0%,fib_23.6%,fib_38.2%,fib_50%,fib_61.8%,fib_76.4%,fib_100%,fib_signal,SentimentScore,Fundamental_Weight,Monthly Return (%),combined_signal,final_trade,Buy Streak,Sell Streak,trade_value
501,2026-01-09,259.08,260.21,256.22,259.37,7.31,AAPL,267.15,273.82,272.58,258.99,232.80,21.94,-3.29,-1.46,8.82,277.84,272.61,269.38,266.77,264.16,260.93,255.70,0,4.75,0.00,1.06,-5.82,HOLD,0,0,0
500,2026-01-08,257.02,259.29,255.70,259.04,10.81,AAPL,268.59,274.40,272.77,258.71,232.62,22.00,-2.84,-1.00,6.72,277.84,272.61,269.38,266.77,264.16,260.93,255.70,0,4.75,0.00,1.06,3.18,HOLD,0,0,0
499,2026-01-07,263.20,263.68,259.81,260.33,10.11,AAPL,269.92,274.97,272.96,258.43,232.42,20.67,-2.20,-0.54,6.62,277.84,272.61,269.38,266.77,264.16,260.93,255.70,0,4.75,0.00,1.06,3.18,HOLD,0,0,0
498,2026-01-06,267.00,267.55,262.12,262.36,11.45,AAPL,270.99,275.34,273.00,258.15,232.21,24.24,-1.48,-0.13,7.48,277.84,272.61,269.38,266.77,264.16,260.93,255.70,0,4.75,0.00,1.06,11.18,HOLD,0,0,0
497,2026-01-05,270.64,271.51,266.14,267.26,9.22,AAPL,272.12,275.47,272.94,257.86,231.96,25.05,-0.74,0.21,13.23,277.84,272.61,269.38,266.77,264.16,260.93,255.70,0,4.75,0.00,1.06,11.18,HOLD,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42067,2024-01-17,228.90,230.33,221.50,225.33,15.92,ZS,0.00,0.00,0.00,0.00,0.00,50.00,0.77,0.42,15.41,231.48,228.51,226.67,225.18,223.70,221.86,218.89,1,2.73,-10.52,1.06,25.18,HOLD,0,0,0
42066,2024-01-16,230.00,233.28,228.78,232.18,8.54,ZS,0.00,0.00,0.00,0.00,0.00,50.00,1.02,0.33,20.93,231.48,228.51,226.67,225.18,223.70,221.86,218.89,0,2.73,-10.52,1.06,17.18,HOLD,0,0,0
42065,2024-01-12,230.98,233.81,229.63,230.77,8.96,ZS,0.00,0.00,0.00,0.00,0.00,50.00,0.62,0.16,21.51,231.48,228.51,226.67,225.18,223.70,221.86,218.89,0,2.73,-10.52,1.06,1.18,HOLD,0,0,0
42064,2024-01-11,226.00,229.91,224.00,228.52,7.47,ZS,0.00,0.00,0.00,0.00,0.00,50.00,0.25,0.05,21.74,231.48,228.51,226.67,225.18,223.70,221.86,218.89,1,2.73,-10.52,1.06,9.18,HOLD,0,0,0


#### Append daily data to Signal DF

In [83]:
signal_df = tableau_df.groupby(["Symbol"]).head(1)

# signal_df.to_csv(r"C:\Users\drcna\OneDrive\Desktop\Stock_Analysis\Stock_Analysis_New\Reports\signals_df.csv", index=False)

old_signal_df = pd.read_csv(r"/Users/drcnavad/Desktop/Stock Analysis/Reports/signal_analysis.csv")
old_signal_df["Date"] = pd.to_datetime(old_signal_df["Date"])

old_signal_df = old_signal_df[old_signal_df["Date"] != signal_df["Date"].iloc[0]]

new_signal_df = pd.concat([signal_df, old_signal_df], ignore_index=True)
new_signal_df.to_csv(r"/Users/drcnavad/Desktop/Stock Analysis/Reports/signal_analysis.csv", index=False)

#### Extra Code for Reference only (not saved!)

In [84]:

x = trade_df.sort_values(by = ["Date", "Symbol"], ascending = [False, False]).drop_duplicates(subset = ["Symbol"])
x = x[["Date", "Close", "Symbol", "combined_signal", "final_trade", "ma_10", "ma_30", "ma_50", "Buy Streak", "Monthly Return (%)", "Fundamental_Weight"]]
x = x.rename(columns = {"combined_signal": "MyScore", "final_trade": "Signal", "Monthly Return (%)": "Sector 2Wk%", "Fundamental_Weight": "BalanceSheetScore"}).reset_index(drop=True)
x = x.sort_values(by = ["MyScore"], ascending = False, ignore_index=True)

print(f"Earnings nearby: {symbols_with_earnings}")
x[(x["Buy Streak"] > 10) & (x["Buy Streak"] < 100)]

Earnings nearby: set()


,Date,Close,Symbol,MyScore,Signal,ma_10,ma_30,ma_50,Buy Streak,Sector 2Wk%,BalanceSheetScore
0,2026-01-09,188.50,RTX,81.90,BUY,186.43,179.72,177.77,56,1.30,8.56
1,2026-01-09,110.53,MRK,79.38,BUY,107.76,103.03,97.91,43,1.46,11.55
2,2026-01-09,187.62,BIIB,75.38,BUY,180.19,177.94,171.39,57,1.46,5.57
3,2026-01-09,345.09,MU,72.18,BUY,313.97,269.82,254.21,93,1.06,14.66
6,2026-01-09,796.55,REGN,70.38,BUY,783.72,759.42,730.41,58,1.46,10.98
7,2026-01-09,247.38,AMZN,70.26,BUY,236.37,231.40,233.32,13,3.42,13.74
8,2026-01-09,32.63,HAL,68.90,BUY,30.04,28.51,27.78,56,1.30,0.52
11,2026-01-09,1400.00,KLAC,59.18,BUY,1310.49,1248.33,1218.64,15,1.06,2.93
12,2026-01-09,1273.88,ASML,59.18,BUY,1161.16,1109.00,1075.79,12,1.06,1.09
15,2026-01-09,218.36,LRCX,57.18,BUY,190.81,172.67,165.41,15,1.06,9.94


In [85]:
symbols = ["META", "ORCL", "TSLA", "AMD", "NVDA", "RTX"]
x[x["Symbol"].isin(symbols)]

,Date,Close,Symbol,MyScore,Signal,ma_10,ma_30,ma_50,Buy Streak,Sector 2Wk%,BalanceSheetScore
0,2026-01-09,188.50,RTX,81.90,BUY,186.43,179.72,177.77,56,1.30,8.56
30,2026-01-09,445.01,TSLA,26.26,HOLD,447.39,454.84,444.26,0,3.42,-4.43
36,2026-01-09,184.86,NVDA,16.18,HOLD,187.60,183.06,186.37,0,1.06,12.24
38,2026-01-09,653.06,META,15.42,HOLD,656.56,654.85,643.07,0,0.14,9.02
60,2026-01-09,203.17,AMD,-15.82,SELL,213.69,214.19,223.98,0,1.06,2.70
76,2026-01-09,198.52,ORCL,-29.82,SELL,194.86,198.88,212.35,0,1.06,7.41
